In [1]:
# !pip install llama-index llama-index-llms-groq groq llama-index-embeddings-huggingface ipywidgets docx2txt

In [1]:
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WORLD_SIZE"] = "1"

In [2]:
from IPython.display import display
import ipywidgets as widgets
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
from llama_index.core import Settings
import warnings
import os

In [3]:
warnings.filterwarnings('ignore')


GROQ_API_KEY = os.getenv("GROQ_API_KEY")

prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

In [ ]:
context = "This directory contains multiple academic documents on large language models (llms) and NLP research"

# Data ingestion: load all files from a directory
directory_path = "/home/olawale/Desktop/PROJECTS/llms/data/input/"  # Update this with your directory path
reader = SimpleDirectoryReader(input_dir=directory_path)
documents = reader.load_data()

# Split the documents into nodes
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

In [5]:
len(documents)

70

In [6]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

In [7]:
Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, node_parser=nodes)
vector_index.storage_context.persist(persist_dir="./storage_mini")

In [9]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")
index = load_index_from_storage(storage_context)

In [6]:
input_box = widgets.Text(
    value='Summarise the Self-Rag paper: Learning to Retrieve, Generate, and Critique through Self-Reflection',
    placeholder='Type your question here',
    description='Question:',
    disabled=False
)

output_area = widgets.Output()

# def on_button_click(b):
#     with output_area:
#         output_area.clear_output()
#         question = input_box.value
#         query_prompt = prompt_template.format(context=context, question=question)
#         resp = query_engine.query(query_prompt).response
#         lines = resp.split("\n")  # Split response by existing lines
#         max_lines = 10
        
#         # Limit to 10 lines or evenly split if fewer natural lines exist
#         if len(lines) > max_lines:
#             lines = lines[:max_lines]
#         else:
#             chunk_size = max(1, len(resp) // max_lines)
#             lines = [resp[i:i+chunk_size] for i in range(0, len(resp), chunk_size)][:max_lines]

#         # Display formatted response
#         for line in lines:
#             print(line)


def on_button_click(b):
    with output_area:
        output_area.clear_output()
        question = input_box.value
        query_prompt = prompt_template.format(context=context, question=question)
        resp = query_engine.query(query_prompt)
        print(resp.response)

button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

button.on_click(on_button_click)

display(input_box, button, output_area)

# Set up query engine
query_engine = index.as_query_engine()

In [ ]:
button.on_click(on_button_click)

display(input_box, button, output_area)

# Set up query engine
query_engine = index.as_query_engine()

In [18]:
# !pip install spacy

In [12]:
from IPython.display import display
import ipywidgets as widgets
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, ServiceContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import warnings
import os
from neo4j import GraphDatabase
import spacy

warnings.filterwarnings('ignore')

# ---- NEO4J SETUP ----
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [13]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# ---- PROMPT TEMPLATE ----
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Graph Insights: {graph_insights}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information and graph insights, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

In [14]:
# Define the context for your prompt
context = "This directory contains multiple academic documents on large language models (llms) and NLP research"

# Data ingestion: load all files from a directory
directory_path = directory_path
reader = SimpleDirectoryReader(input_dir=directory_path)
documents = reader.load_data()

# Load spacy model (you can choose a different model)
nlp = spacy.load("en_core_web_sm")

In [9]:
def populate_graph(documents, driver, nlp):
    with driver.session() as session:
        for doc in documents:
            doc_text = doc.text  # Assuming each document has a 'text' attribute
            nlp_doc = nlp(doc_text)
            # concepts = [ent.text for ent in nlp_doc.ents if ent.label_ == "ORG" or ent.label_ == "CARDINAL"] # Adjust entity types as needed
            # concepts = [ent.text for ent in nlp_doc.ents] # Adjust entity types as needed
            concepts = [ent.text for ent in nlp_doc.ents if ent.label_ == "ORG" or ent.label_ == "PRODUCT"] # Adjust entity types as needed
            # print("concepts", concepts)
            # for ent in nlp_doc.ents:
            #   print(ent.label_)

            for concept in concepts:
                # print("concept", concept)
                session.run("MERGE (:Concept {name: $concept})", concept=concept)

            for i, concept in enumerate(concepts):
                if i + 1 < len(concepts):
                    next_concept = concepts[i + 1]
                    session.run(
                        """
                        MATCH (c1:Concept {name: $concept}), (c2:Concept {name: $next_concept})
                        MERGE (c1)-[:RELATED_TO]->(c2)
                        """,
                        concept=concept, next_concept=next_concept
                    )

# Populate the Neo4j graph
populate_graph(documents, driver, nlp)

In [ ]:
# Split the documents into nodes
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

# Set up embedding model and LLM
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

In [ ]:
Settings.llm = llm
Settings.embed_model = embed_model
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, node_parser=nodes)
vector_index.storage_context.persist(persist_dir="./storage_mini")
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")
index = load_index_from_storage(storage_context)

In [17]:
# Create the interactive widgets
input_box = widgets.Text(
    value='Summarise the Self-Rag paper: Learning to Retrieve, Generate, and Critique through Self-Reflection',
    placeholder='Type your question here',
    description='Question:',
    disabled=False
)

output_area = widgets.Output()

In [18]:
#Query Enhancement with Neo4j

def get_graph_insights(question):
  with driver.session() as session:
    result = session.run(
         """
            MATCH (c:Concept)
            WHERE toLower(c.name) CONTAINS toLower($question)
            OPTIONAL MATCH (c)-[r:RELATED_TO]->(other:Concept)
            RETURN c.name AS concept, collect(other.name) AS related_concepts
            """,
         question=question
         )
    insights = []
    for record in result:
       insights.append(f"Concept: {record['concept']}, Related Concepts: {', '.join(record['related_concepts'])}")
       return "\n".join(insights) if insights else "No relevant graph insights found."

In [61]:
type(get_graph_insights("self rag "))

NoneType

In [ ]:
# def on_button_click(b):
#     with output_area:
#         output_area.clear_output()
#         question = input_box.value
#         query_prompt = prompt_template.format(context=context, question=question)
#         resp = query_engine.query(query_prompt).response
#         lines = resp.split("\n")  # Split response by existing lines
#         max_lines = 10
        
#         # Limit to 10 lines or evenly split if fewer natural lines exist
#         if len(lines) > max_lines:
#             lines = lines[:max_lines]
#         else:
#             chunk_size = max(1, len(resp) // max_lines)
#             lines = [resp[i:i+chunk_size] for i in range(0, len(resp), chunk_size)][:max_lines]

#         # Display formatted response
#         for line in lines:
#             print(line)

def on_button_click(b):
    with output_area:
        output_area.clear_output()
        question = input_box.value
        query_prompt = prompt_template.format(context=context, question=question)
        resp = query_engine.query(query_prompt)
        print(resp.response)
        
button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

button.on_click(on_button_click)

display(input_box, button, output_area)

#Query Engine Setup
query_engine = index.as_query_engine()

**Summary of the Self-RAG Paper**

The Self-RAG paper introduces a new framework to enhance the quality and factuality of Large Language Models (LLMs) through retrieval on demand and self-reflection. The framework trains an LLM to learn to retrieve, generate, and critique text passages and its own generation by predicting the next tokens from its original vocabulary as well as newly added special tokens, called reflection tokens. These reflection tokens signal the need for retrieval or confirm the output's relevance, support, or completeness.

**Additional Helpful Information**

The paper highlights that common RAG approaches retrieve passages indiscriminately, without ensuring complete support from cited sources. In contrast, Self-RAG enables the tailoring of LLM behaviors at test time by leveraging reflection tokens. The authors also claim that their holistic evaluations on six t

In [1]:
button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

button.on_click(on_button_click)

display(input_box, button, output_area)

#Query Engine Setup
query_engine = index.as_query_engine()